# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Изучение данных и общей информации

In [1]:
import pandas as pd
data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')
data.info() #просматриваем общую информацию о загруженной базе данных

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 21525 entries, (1,-8437, 673027760233,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875) to (2,-1984, 5075885305268,40,среднее,1,женат / замужем,0,F,сотрудник,0,82047)
Data columns (total 1 columns):
children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose    19351 non-null object
dtypes: object(1)
memory usage: 519.5+ KB


In [2]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Вывод**

База данных состоит из 12-ти столбцов различных типов данных (числовые целые, дробные и текстовые) и содержит 21525 записей; отдельные ячейки таблицы не заполнены

## Предобработка данных

### Обработка пропусков

- Заполнение пропусков days_employed

In [3]:
data.loc[data['days_employed'].isna(), 'days_employed'] = data['days_employed'].median() #  производится заполнение количественных
#  данных медианными значениями по столбцу days_employed

#  data.loc[data['total_income'].isna(), 'total_income'] = data.groupby('income_type')['total_income'].mean()#  производится заполнение количественных
#  данных медианными значениями по столбцу total_income

- Заполнение пропусков total_income

In [4]:
for income in data['income_type'].unique(): #  с помощью цикла по уникальным значениям столбца income_type
    median = data.loc[data['income_type'] == income, 'total_income'].median()  #  вычисляем для каждой категории медианное значение
    data.loc[(data['total_income'].isna()) & (data['income_type'] == income), 'total_income'] = median #  присваиваем пропущенным 
    #  полям total_income значения медианы в cоответствии с категорией по столбцу income_type 
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


**Вывод**

Пропуски в полях days_employed и total_income в исходной базе данных носят произвольный характер, распределение категорий с незаполненными данными соответствует распределению в общей выборке, поэтому отсутствующие значения количественных переменных в полях заменены на медианные **(по столбцу total_income медианные значения присвоены с учетом категоризации по столбцу income_type)**

In [5]:
data['dob_years'].value_counts().reset_index().sort_values(by = 'index').head() 
#  формируем выборку записей по возрасту с сортировкой по возрастанию

,index,dob_years
47,0,101
54,19,14
52,20,51
46,21,111
44,22,183


### Замена типа данных

In [6]:
data['days_employed'] = data['days_employed'].astype('int').abs() #данные в столбце days_employed по своему смыслу
#соответствуют типу данных целое число, также избавляемся от отрицательных значений оператором abs()
data['total_income'] = data['total_income'].round(2) #округляем до сотых
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу


**Вывод**

данные в столбце days_employed по своему смыслу соответствуют типу данных целое число, поэтому переведены в целое число

### Обработка дубликатов

In [7]:
data['education'].value_counts()
data['education'] = data['education'].str.lower() #преобразование данных в столбце к строчному написанию
data['family_status'] = data['family_status'].str.lower()

In [8]:
data.duplicated().sum() #  выводим сумму строк-дубликатов

71

In [9]:
data.drop_duplicates().reset_index(drop = True).info() #  удаляем дубликаты, проводим переиндексацию полей

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21454 entries, 0 to 21453
Data columns (total 12 columns):
children            21454 non-null int64
days_employed       21454 non-null int64
dob_years           21454 non-null int64
education           21454 non-null object
education_id        21454 non-null int64
family_status       21454 non-null object
family_status_id    21454 non-null int64
gender              21454 non-null object
income_type         21454 non-null object
debt                21454 non-null int64
total_income        21454 non-null float64
purpose             21454 non-null object
dtypes: float64(1), int64(6), object(5)
memory usage: 2.0+ MB


**Вывод**

дубликаты выявлены с помощью функции value_counts(), появдение дубликатов вызваны влиянием человеческого фактора 
дубликаты в столбце education через приведение к строчным буквам исключены, **проведено удаление дубликатов, выборка сократилась на 71 строку**

### Лемматизация

In [10]:
data['purpose'].value_counts()
from pymystem3 import Mystem #  загружаем библиотеку
m = Mystem()
purpose_lemmas = ' '.join(data['purpose'])  #  преобразуем данные из столбца в строку
print_lemmas = m.lemmatize(purpose_lemmas)

from collections import Counter #  импортируем контейнер для подсчета значений
print(Counter(print_lemmas))

Counter({' ': 55201, 'недвижимость': 6367, 'покупка': 5912, 'жилье': 4473, 'автомобиль': 4315, 'образование': 4022, 'с': 2924, 'операция': 2610, 'свадьба': 2348, 'свой': 2235, 'на': 2233, 'строительство': 1881, 'высокий': 1375, 'получение': 1316, 'коммерческий': 1315, 'для': 1294, 'жилой': 1233, 'сделка': 944, 'дополнительный': 909, 'заниматься': 908, 'подержать': 858, 'проведение': 777, 'сыграть': 774, 'сдача': 653, 'семья': 641, 'собственный': 635, 'со': 630, 'ремонт': 612, 'приобретение': 462, 'профильный': 436, 'подержанный': 110, '\n': 1})


**Вывод**

подсчет лемматизированных данных по столбцу purpose позволяет выделить следующие основные цели поличения кредита: недвижимость, жилье, автомобиль, образование, свадьба, ремонт

### Категоризация данных

In [11]:
def purpose_group(purpose):  #  объявление функции присвоения категорий
    purpose_lemmas = m.lemmatize(purpose)
    if 'ремонт' in purpose_lemmas:
        return 'ремонт'
    if ('недвижимость' in purpose_lemmas) or ('жилье' in purpose_lemmas):
        return 'недвижимость'
    if 'автомобиль' in purpose_lemmas:
        return 'автомобиль'
    if 'образование' in purpose_lemmas:
        return 'образование'
    if 'свадьба' in purpose_lemmas:
        return 'свадьба'
data['purpose_group'] = data['purpose'].apply(purpose_group) #  создание столбца из данных полученных применением 
#  функции purpose_group
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_group
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.64,покупка жилья,недвижимость
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.01,приобретение автомобиля,автомобиль
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.95,покупка жилья,недвижимость
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.55,дополнительное образование,образование
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.08,сыграть свадьбу,свадьба


**Вывод**

выполнена категоризация по целям получения кредита, добавлен дополнительный столбец

## Проверка предположений

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [12]:
data['children'] = data['children'].replace(20, 2) #  замена артефакта данных в столбце 
data['children'] = data['children'].abs() #  замена артефакта данных в столбце путем принятия по значения по модулю

In [13]:
#  формируем сводную таблицу из оценки влияния критериев на показатель
children_pivot = data.pivot_table(index = ['children'], columns = 'debt', values = 'total_income', aggfunc = 'count') 
children_pivot['ratio, %'] = (children_pivot[1] / (children_pivot[0] + children_pivot[1]) * 100).round(1) #  добавляем столбец процентного соотношения
children_pivot

debt,0,1,"ratio, %"
children,,,
0,13086.0,1063.0,7.5
1,4420.0,445.0,9.1
2,1929.0,202.0,9.5
3,303.0,27.0,8.2
4,37.0,4.0,9.8
5,9.0,NaN,NaN


**Вывод**

Наличие детей повышает вероятность нарушения графика платежей: доля заемщиков без детей допускающих просрочки - 7,5%, с детьми - **8-10%** 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [14]:
#  формируем сводную таблицу из оценки влияния критериев на показатель
family_pivot = data.pivot_table(index = ['family_status'], columns = 'debt', values = 'total_income', aggfunc = 'count') 
family_pivot['ratio, %'] = (family_pivot[1] / (family_pivot[0] + family_pivot[1]) * 100).round(1) #  добавляем столбец процентного соотношения
family_pivot.sort_values(by = 'ratio, %') #  добавляем сортировку по возрастанию

debt,0,1,"ratio, %"
family_status,,,
вдовец / вдова,897,63,6.6
в разводе,1110,85,7.1
женат / замужем,11449,931,7.5
гражданский брак,3789,388,9.3
не женат / не замужем,2539,274,9.7


**Вывод**

Заемщики, вступившие в официальный брак (в том числе, находившиеся в официальном браке ранее) более ответственно соблюдают график платежей, чем находящиеся в гражданском браке или неженатые и незамужние

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [15]:
def income_grade(income): #  категоризация данных столбца total_income с помощью функции income_grade(income)
    income_mean = data['total_income'].mean() #  категории сформированы относительно среднего значения выборки
    if income <= income_mean / 2: 
        return 'низкий'
    if income <= income_mean:
        return 'ниже среднего'
    if income <= income_mean * 1.5:
        return 'выше среднего'    
    if income > income_mean * 1.5:
        return 'высокий'
income_group = data.loc[:, ('total_income', 'debt')]   #  создаем новую таблицу для формирования для работы с новым столбцом
income_group['income_grade'] = income_group['total_income'].apply(income_grade)  #  создаем столбец из критериев уровня доходов
#  формируем сводную таблицу из оценки влияния критериев на показатель
income_pivot = income_group.pivot_table(index = ['income_grade'], columns = 'debt', values = 'total_income', aggfunc = 'count') 
income_pivot['ratio, %'] = (income_pivot[1] / (income_pivot[0] + income_pivot[1]) * 100).round(1) #  добавляем столбец процентного соотношения
income_pivot.sort_values(by = 'ratio, %') #  добавляем сортировку по возрастанию

debt,0,1,"ratio, %"
income_grade,,,
высокий,2685,198,6.9
низкий,2358,196,7.7
выше среднего,4879,426,8.0
ниже среднего,9862,921,8.5


**Вывод**

Заемщики с самыми высокими доходами менее склонны нарушать график платежей по кредиту 

- Как разные цели кредита влияют на его возврат в срок?

In [16]:
purpose_pivot = data.pivot_table(index = ['purpose_group'], columns = 'debt', values = 'total_income', aggfunc = 'count') 
purpose_pivot['ratio, %'] = (purpose_pivot[1] / (purpose_pivot[0] + purpose_pivot[1]) * 100).round(1) #  добавляем столбец процентного соотношения
purpose_pivot.sort_values(by = 'ratio, %') #  добавляем сортировку по возрастанию

debt,0,1,"ratio, %"
purpose_group,,,
ремонт,577,35,3.0
недвижимость,9481,747,3.9
свадьба,2162,186,4.3
образование,3652,370,5.1
автомобиль,3912,403,5.2


**Вывод**

Наибольший процент просрочек происходит по кредитам, выданным на покупку автомобиля и получение образования

## Общий вывод

Наименьшая вероятность нарушения кредитной дисциплины у заемщиков без детей, находящиеся (находившиеся) в официальном браке, с высоким доходом по кредитам с целью **проведения ремонта или** приобретения недвижимости; наибольшая вероятность у заемщиком, имеющих детей, не находящиеся в офицальном браке со средними доходами по кредитам на приобретение автомобиля или образование.
**В качестве развития исследования целесообразно рассмотреть влияние пола и типа занятости** 